In [ ]:
!pip install groq --quiet
import os
from groq import Groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 10.1 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
client = Groq(
    api_key=userdata.get('Groq'),
)

In [ ]:
from typing import List, Dict, Optional
from IPython.display import display, Markdown

In [ ]:
class ReflectionAgent:

    def __init__(self, client):
        self.client = client
        self.model = "llama-3.1-8b-instant"
        self.generator_history: List[Dict] = []
        self.reflection_history: List[Dict] = []

    def generate_code(self,user_query: str,system_prompt: Optional[str] = None, max_iterations: int = 2) -> Dict[str, any]:

        default_generator_prompt = """
        You are an Python programmer.
        Generate code based on user requests. Do NOT provide any explanations of your code, just simply the necessary python code.
        """

        default_reflection_prompt = """
        You are a senior code reviewer.

        Review the code and provide brief, actionable feedback.

        Do NOT include any code examples. Keep the feedback concise within 2/3 lines.
        If everything looks good, reply only: "passed".

        """

        self.generator_history = [
            {"role": "system", "content":  default_generator_prompt}
        ]
        self.reflection_history = [
            {"role": "system", "content": default_reflection_prompt}
        ]

        self.generator_history.append({"role": "user", "content": user_query})

        current_code = self._call_generator()
        display(Markdown(current_code))

        for iteration in range(max_iterations):
            print(f"\n{'='*60}")
            print(f"Iteration {iteration + 1}/{max_iterations}")
            print(f"{'='*60}\n")

            feedback = self._get_reflection(current_code)
            print(f"\033[33m Received Feedback from Evaluator Agent: \n {feedback}\033[0m" )

            if self._is_code_satisfactory(feedback):
                print("Code deemed satisfactory. Stopping iterations.")
                break

            current_code = self._improve_code(feedback)
            display(Markdown(current_code))

        return {
            "final_code": current_code,
            "iterations": min(iteration + 1, max_iterations),
            "generator_history": self.generator_history,
            "reflection_history": self.reflection_history
        }

    def _call_generator(self) -> str:
        """Call the generator model and return the response."""
        response = self.client.chat.completions.create(
            model=self.model,
            messages=self.generator_history
        )
        code = response.choices[0].message.content
        self.generator_history.append({"role": "assistant", "content": code})
        return code

    def _get_reflection(self, code: str) -> str:
        """Get reflection feedback on the generated code."""
        # Add code to reflection history
        self.reflection_history.append({
            "role": "user",
            "content": f"Please review this code and provide feedback:\n\n{code}"
        })

        # Get reflection
        response = self.client.chat.completions.create(
            model=self.model,
            messages=self.reflection_history
        )
        feedback = response.choices[0].message.content
        self.reflection_history.append({"role": "assistant", "content": feedback})

        return feedback

    def _improve_code(self, feedback: str) -> str:
        """Improve code based on reflection feedback."""
        self.generator_history.append({
            "role": "user",
            "content": f"Please improve the code based on this feedback:\n\n{feedback}"
        })
        return self._call_generator()

    def _is_code_satisfactory(self, feedback: str) -> bool:
        satisfactory_indicators = [
          "Passed"
        ]
        feedback_lower = feedback.lower()
        return any(indicator in feedback_lower for indicator in satisfactory_indicators)



In [ ]:
agent = ReflectionAgent(client)
agent.generate_code("Create a python code to calculate the sum of 1 to n, given input n")['iterations']

```python
def sum_n(n):
    return sum(range(1, n + 1))

n = int(input("Enter a number: "))
print(f"The sum of 1 to {n} is: {sum_n(n)}")
```


Iteration 1/2

 Received Feedback from Evaluator Agent: 
 Input validation is missing. Consider adding checks to ensure user input is a positive integer.

Consider using a more descriptive variable name instead of `n`.


```python
def calculate_sum_of_n():
    while True:
        try:
            num = int(input("Enter a positive integer: "))
            if num > 0:
                return num
            else:
                print("Please enter a positive integer.")
        except ValueError:
            print("Invalid input. Please enter an integer.")

def sum_of_numbers(num):
    return sum(range(1, num + 1))

num = calculate_sum_of_n()
print(f"The sum of 1 to {num} is: {sum_of_numbers(num)}")
```


Iteration 2/2

 Received Feedback from Evaluator Agent: 
 Variable `num` in the `sum_of_numbers` function is unnecessary given its single use and can be directly used within the function.

Function naming conventions could be followed, e.g. `calculate_sum_of_n` could be `calculate_sum`.


```python
def calculate_sum():
    while True:
        try:
            num = int(input("Enter a positive integer: "))
            if num > 0:
                return num
            else:
                print("Please enter a positive integer.")
        except ValueError:
            print("Invalid input. Please enter an integer.")

def sum_range_to_n(n):
    return sum(range(1, n + 1))

calculated_sum = sum_range_to_n(calculate_sum())
print(f"The sum of 1 to {calculated_sum} is: {calculated_sum}")
```

2

In [ ]:
agent = ReflectionAgent(client)
agent.generate_code("Write a code to generate 5 fibonacci numbers")['iterations']

```python
def fibonacci(n):
    fib_sequence = [0, 1]
    while len(fib_sequence) < n:
        fib_sequence.append(fib_sequence[-1] + fib_sequence[-2])
    return fib_sequence[:n]

print(fibonacci(5))
```


Iteration 1/2

 Received Feedback from Evaluator Agent: 
 Handle potential edge cases for n, such as n = 0 or n < 0.


```python
def fibonacci(n):
    if not isinstance(n, int):
        raise TypeError("n must be an integer")
    if n < 0:
        raise ValueError("n must be a non-negative integer")
    if n == 0:
        return []
    elif n == 1:
        return [0]
    else:
        fib_sequence = [0, 1]
        while len(fib_sequence) < n:
            fib_sequence.append(fib_sequence[-1] + fib_sequence[-2])
        return fib_sequence

print(fibonacci(5))
```


Iteration 2/2

 Received Feedback from Evaluator Agent: 
 Consider adding a docstring to explain the function's purpose and behavior. 

Consider adding a check to prevent unnecessary computation for n = 1.


```python
def fibonacci(n):
    """
    Generate the first n Fibonacci numbers.

    Args:
        n (int): The number of Fibonacci numbers to generate.

    Returns:
        list: A list of the first n Fibonacci numbers.

    Raises:
        TypeError: If n is not an integer.
        ValueError: If n is a negative integer.

    """
    if not isinstance(n, int):
        raise TypeError("n must be an integer")
    if n < 0:
        raise ValueError("n must be a non-negative integer")
    if n == 0:
        return []
    elif n == 1:
        return [0]  # Handle edge case where n = 1 to prevent unnecessary computation
    else:
        fib_sequence = [0, 1]
        while len(fib_sequence) < n:
            fib_sequence.append(fib_sequence[-1] + fib_sequence[-2])
        return fib_sequence

print(fibonacci(5))
```

2

In [ ]:
agent = ReflectionAgent(client)
agent.generate_code("Given a sentence try to find whether that is a palindrome or not")['iterations']

```python
def is_palindrome(sentence):
    sentence = ''.join(e for e in sentence if e.isalnum()).lower()
    return sentence == sentence[::-1]

sentence = input("Enter a sentence: ")
print(is_palindrome(sentence))
```

```python
def is_palindrome(sentence):
    sentence = ''.join(e for e in sentence if e.isalnum()).lower()
    return sentence == sentence[::-1]

# Test the function
sentences = ["Madam", "hello", "A man, a plan, a canal, Panama", "Not a palindrome"]
for sentence in sentences:
    print(f"{sentence}: {is_palindrome(sentence)}")
```


Iteration 1/2

 Received Feedback from Evaluator Agent: 
 For the first code snippet:
- Consider adding a docstring to explain the function's purpose and behavior.
- Input validation would be beneficial to handle edge cases such as empty input.

For the second code snippet:
- The test cases seem comprehensive, but consider adding a non-English sentence to test the function's handling of non-ASCII characters.
- The output formatting could be improved for better readability.


```python
def is_palindrome(sentence):
    """
    Checks if a given sentence is a palindrome.

    A palindrome is a string that reads the same backwards as forwards.
    Non-alphanumeric characters and case are ignored.

    Args:
        sentence (str): The input sentence to check.

    Returns:
        bool: True if the sentence is a palindrome, False otherwise.

    Raises:
        ValueError: If the input is not a string.
        TypeError: If the input contains non-string-like elements.
    """
    if not isinstance(sentence, str):
        raise ValueError("Input must be a string.")
    sentence = ''.join(e for e in sentence if e.isalnum()).lower()
    return sentence == sentence[::-1]

sentence = input("Enter a sentence: ")
if sentence:
    print(f"Is '{sentence}' a palindrome? {is_palindrome(sentence)}")
else:
    print("Please enter a non-empty sentence.")
```

```python
def is_palindrome(sentence):
    """
    Checks if a given sentence is a palindrome.

    A palindrome is a string that reads the same backwards as forwards.
    Non-alphanumeric characters and case are ignored.

    Args:
        sentence (str): The input sentence to check.

    Returns:
        bool: True if the sentence is a palindrome, False otherwise.

    Raises:
        ValueError: If the input is not a string.
        TypeError: If the input contains non-string-like elements.
    """
    if not isinstance(sentence, str):
        raise ValueError("Input must be a string.")
    sentence = ''.join(e for e in sentence if e.isalnum()).lower()
    return sentence == sentence[::-1]

# Test the function
sentences = [
    "Madam",
    "hello",
    "A man, a plan, a canal, Panama",
    "Not a palindrome",
    "Kajak",
    "Radar",
    "¡Hola!",
    "Bonjour!",
    "",
]

for i, sentence in enumerate(sentences):
    try:
        result = is_palindrome(sentence)
        print(f"{i+1}. '{sentence}': {result}")
    except (ValueError, TypeError) as e:
        print(f"{i+1}. '{sentence}': {e}")
``````


Iteration 2/2

 Received Feedback from Evaluator Agent: 
 passed


```python
def is_palindrome(sentence):
    """
    Checks if a given sentence is a palindrome.

    A palindrome is a string that reads the same backwards as forwards.
    Non-alphanumeric characters and case are ignored.

    Args:
        sentence (str): The input sentence to check.

    Returns:
        bool: True if the sentence is a palindrome, False otherwise.

    Raises:
        ValueError: If the input is not a string.
        TypeError: If the input contains non-string-like elements.
    """
    if not isinstance(sentence, str):
        raise ValueError("Input must be a string.")
    sentence = ''.join(e for e in sentence if e.isalnum()).lower()
    return sentence == sentence[::-1]

def main():
    while True:
        sentence = input("Enter a sentence (or 'quit' to exit): ")
        if sentence.lower() == 'quit':
            break
        if sentence:
            try:
                result = is_palindrome(sentence)
                print(f"Is '{sentence}' a palindrome? {result}")
            except (ValueError, TypeError) as e:
                print(f"Error: {e}")
        else:
            print("Please enter a non-empty sentence.")

if __name__ == "__main__":
    main()
```

```python
def is_palindrome(sentence):
    """
    Checks if a given sentence is a palindrome.

    A palindrome is a string that reads the same backwards as forwards.
    Non-alphanumeric characters and case are ignored.

    Args:
        sentence (str): The input sentence to check.

    Returns:
        bool: True if the sentence is a palindrome, False otherwise.

    Raises:
        ValueError: If the input is not a string.
        TypeError: If the input contains non-string-like elements.
    """
    if not isinstance(sentence, str):
        raise ValueError("Input must be a string.")
    sentence = ''.join(e for e in sentence if e.isalnum()).lower()
    return sentence == sentence[::-1]

def main():
    test_sentences = [
        "Madam",
        "hello",
        "A man, a plan, a canal, Panama",
        "Not a palindrome",
        "Kajak",
        "Radar",
        "¡Hola!",
        "Bonjour!",
        "",
    ]

    print("Palindrome Test Results:")
    for i, sentence in enumerate(test_sentences):
        try:
            result = is_palindrome(sentence)
            print(f"{i+1}. '{sentence}': {result}")
        except (ValueError, TypeError) as e:
            print(f"{i+1}. '{sentence}': {e}")

if __name__ == "__main__":
    main()
```

2

In [ ]:
agent = ReflectionAgent(client)
agent.generate_code("Write a Python function that generates a random password")['iterations']

```python
import random
import string

def generate_password(length=12):
    characters = string.ascii_letters + string.digits + string.punctuation
    password = ''.join(random.choice(characters) for _ in range(length))
    return password

print(generate_password())
```


Iteration 1/2

 Received Feedback from Evaluator Agent: 
 * Use a specific seed for the random number generator for reproducibility.
* Consider adding error handling for invalid `length` values.
* Add docstrings for `generate_password` function.
* Return a `bytes` representation to include non-ASCII characters.

```python
import random
import string

def generate_password(length=12, seed=None):
#    ...
```


```python
import random
import string

def generate_password(length=12, seed=None):
    """
    Generates a random password.

    Args:
        length (int): The length of the password. Defaults to 12.
        seed (int): A seed for the random number generator. Defaults to None.

    Returns:
        bytes: A random password as a bytes object.

    Raises:
        ValueError: If length is less than 1.
    """
    if seed is not None:
        random.seed(seed)
    if length < 1:
        raise ValueError("Length must be at least 1")

    characters = string.ascii_letters + string.digits + string.punctuation
    password = ''.join(random.choice(characters) for _ in range(length))
    return password.encode('utf-8')

print(generate_password())
```


Iteration 2/2

 Received Feedback from Evaluator Agent: 
 * Document the characters used for the password.
* Consider adding a parameter to control the proportion of each character type in the password.
* Handle the case where `length` is too large for the available characters.
* Use the `random` module's `choices` function for generating the password.


```python
import random
import string

def generate_password(length=12, seed=None, ascii_weight=0.65, digit_weight=0.2, punctuation_weight=0.15):
    """
    Generates a random password.

    Args:
        length (int): The length of the password. Defaults to 12.
        seed (int): A seed for the random number generator. Defaults to None.
        ascii_weight (float): Proportion of ASCII letters in the password (0.0 to 1.0). Defaults to 0.65.
        digit_weight (float): Proportion of digits in the password (0.0 to 1.0). Defaults to 0.2.
        punctuation_weight (float): Proportion of punctuation characters in the password (0.0 to 1.0). Defaults to 0.15.

    Returns:
        bytes: A random password as a bytes object.

    Raises:
        ValueError: If length is less than 1 or if the proportion of the three character types exceeds 1.0.
    """
    if seed is not None:
        random.seed(seed)
    if length < 1:
        raise ValueError("Length must be at least 1")
    if (ascii_weight + digit_weight + punctuation_weight) > 1.0:
        raise ValueError("Proportion of character types cannot exceed 1.0")

    num_ascii = int(length * ascii_weight)
    num_digits = int(length * digit_weight)
    num_punctuation = length - num_ascii - num_digits
    characters = {
        'ascii': [char for char in string.ascii_letters if char.isascii()],
        'digits': [char for char in string.digits],
        'punctuation': [char for char in string.punctuation]
    }
    password = []
    for _ in range(num_ascii):
        password.append(random.choice(characters['ascii']))
    for _ in range(num_digits):
        password.append(random.choice(characters['digits']))
    for _ in range(num_punctuation):
        password.append(random.choice(characters['punctuation']))

    # Fill the rest of the password with randomly chosen characters
    extra_characters = sorted([char for char_type in characters.values() for char in char_type])
    while len(password) < length:
        password.append(random.choice(extra_characters))

    # Shuffle the password to remove any bias from the order of generation
    random.shuffle(password)
    password = ''.join(password)
    return password.encode('utf-8')

print(generate_password())
```

2